IMPORTAÇÃO DE MODULOS

In [67]:
from path_arquivos import *
import pandas as pd
import numpy as np
import glob
import os

EXTRAÇÃO

In [68]:
PATH_AE = r'c:\WS_OLIVEIRA\BASE_DADOS\1702 - Cadastro de Endereços.txt'
COL_AE = ['COD_END', 'DEP', 'RUA', 'PREDIO', 'NIVEL', 'APTO', 'STATUS', 'ROTATIVO', 'BLOQ', 'DISP', 'PL', 'STR']
ae = pd.read_csv(PATH_AE, header= None, names= COL_AE)
pk_end = pd.DataFrame({"codigo": [1,2,3,4,5,6,7,8],
                       "desc": ['1-INTEIRO (2,55)','2-INTEIRO(1,90)','3-MEDIO (0,80)', '4-TERCO (0,56)', '5-TERCO (0,46)', '6-PRATELEIRA', '7-MEIO PALETE', '8-INTEIRO(1,35)']})

CORPO DO CODIGO

In [ ]:

df_aereos = ae.merge(pk_end, left_on='PL', right_on= 'codigo', how='left').drop(columns=['codigo', ])

maior_pd = df_aereos.groupby([df_aereos['RUA'].astype(str) +"-"+ df_aereos['PREDIO'].astype(str)])['NIVEL'].max().reset_index()
maior_pd.columns = ['CONCAT', 'maior_nivel']
df_aereos['CONCAT'] = df_aereos['RUA'].astype(str) +"-"+ df_aereos['PREDIO'].astype(str)
df_aereos = df_aereos.merge(maior_pd, left_on= 'CONCAT', right_on= 'CONCAT', how= 'left')

df_aereos['VALIDAR'] = np.where(df_aereos['maior_nivel'] == df_aereos['NIVEL'], 
                                np.where(df_aereos['desc'] != '8-INTEIRO(1,35)', "incorreto", "correto"),
                                np.where(df_aereos['desc'] == '8-INTEIRO(1,35)', "incorreto", "correto")
)


CONSULTAS

In [70]:
display(df_aereos.loc[df_aereos['VALIDAR'] == 'incorreto'])
retur = df_aereos.loc[(df_aereos['VALIDAR'] == 'incorreto') & (~df_aereos['RUA'].isin([23,24,25]))]

,COD_END,DEP,RUA,PREDIO,NIVEL,APTO,STATUS,ROTATIVO,BLOQ,DISP,PL,STR,desc,CONCAT,maior_nivel,VALIDAR
15116,1036089,1,23,2,3,301,PULMÃO,N,NÃO,OCUPADO,8,1,"8-INTEIRO(1,35)",23-2,5,incorreto
15117,1036090,1,23,2,3,302,PULMÃO,N,NÃO,OCUPADO,8,1,"8-INTEIRO(1,35)",23-2,5,incorreto
15118,1039647,1,23,2,4,401,PULMÃO,N,NÃO,OCUPADO,8,1,"8-INTEIRO(1,35)",23-2,5,incorreto
15119,1039648,1,23,2,4,402,PULMÃO,N,NÃO,OCUPADO,8,1,"8-INTEIRO(1,35)",23-2,5,incorreto
15130,1034315,1,23,4,2,201,PULMÃO,N,NÃO,OCUPADO,8,1,"8-INTEIRO(1,35)",23-4,6,incorreto
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29827,1063920,1,202,22,5,502,PULMÃO,N,NÃO,LIVRE,2,1,"2-INTEIRO(1,90)",202-22,5,incorreto
29834,1063921,1,202,23,5,501,PULMÃO,N,NÃO,LIVRE,2,1,"2-INTEIRO(1,90)",202-23,5,incorreto
29841,1063924,1,202,24,5,502,PULMÃO,N,NÃO,LIVRE,2,1,"2-INTEIRO(1,90)",202-24,5,incorreto
29842,1063935,1,202,30,5,501,PULMÃO,N,NÃO,LIVRE,3,1,"3-MEDIO (0,80)",202-30,5,incorreto


In [71]:
retorno = r'c:\WS_OLIVEIRA\RESULTADO.xlsx'
retur.to_excel(retorno,index= False, sheet_name= 'retorno')